In [ ]:
!pip install matrixprofile-ts
! pip install pyts


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import matrixprofile as mp
from matrixprofile import *
from matrixprofile.discords import discords
from pyts.approximation import PiecewiseAggregateApproximation

     |████████████████████████████████| 2.5MB 3.4MB/s 


In [ ]:
df1=pd.read_csv('/content/sample_data/us-counties-iso.csv')


#Selected Baltimore County for the analysis

df1_B = df1[df1["county"] == "Baltimore"]
df1_B_groupby = df1_B.groupby("date").sum()
cases_B = df1_B_groupby["cases"].values

new_dailycases_B = []
new_dailycases_B.append(cases_B[0])

for i in range(len(cases_B)-1):
    num_cases = cases_B[i+1] - cases_B[i]
    new_dailycases_B.append(num_cases)

new_dailycases_B = np.array(new_dailycases_B)
new_dailycases_B
#print(new_dailycases_B)
#print(len(new_dailycases_B))


#Adding a new column to the existing dataframe
df1_B_groupby["New Daily Cases"] = new_dailycases_B


#Converting above dataframe to a CSV file
df1_B_groupby.to_csv("B_1 Non Cumu Data.csv")

In [ ]:
df_converted = pd.read_csv('B_1 Non Cumu Data.csv')
model=IsolationForest(n_estimators=50, max_samples='auto', contamination=float(0.1),max_features=1.0)
model.fit(df_converted[['New Daily Cases']])

IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.1,
                max_features=1.0, max_samples='auto', n_estimators=50,
                n_jobs=None, random_state=None, verbose=0, warm_start=False)

In [ ]:
df_converted['scores']=model.decision_function(df_converted[['New Daily Cases']])
df_converted['anomaly']=model.predict(df_converted[['New Daily Cases']])
df_converted.head(118)

,date,fips,cases,deaths,New Daily Cases,scores,anomaly
0,2020-03-11,24005.0,1,0,1,0.055418,1
1,2020-03-12,24005.0,1,0,0,0.034180,1
2,2020-03-13,24005.0,2,0,1,0.055418,1
3,2020-03-14,24005.0,3,0,1,0.055418,1
4,2020-03-15,24005.0,3,0,0,0.034180,1
...,...,...,...,...,...,...,...
113,2020-07-02,24005.0,8126,474,64,0.120807,1
114,2020-07-03,24005.0,8204,478,78,0.075651,1
115,2020-07-04,24005.0,8255,480,51,0.122133,1
116,2020-07-05,24005.0,8306,481,51,0.122133,1


In [ ]:
anomaly=df_converted.loc[df_converted['anomaly']==-1]
anomaly_index=list(anomaly.index)
print(len(anomaly))
print(anomaly)

11
          date     fips  cases  deaths  New Daily Cases    scores  anomaly
28  2020-04-08  24005.0    866      13              214 -0.191842       -1
45  2020-04-25  24005.0   2397      79              154 -0.041744       -1
51  2020-05-01  24005.0   3026     129              182 -0.139562       -1
52  2020-05-02  24005.0   3197     139              171 -0.078253       -1
54  2020-05-04  24005.0   3463     151              148 -0.012403       -1
55  2020-05-05  24005.0   3445     157              -18 -0.160358       -1
59  2020-05-09  24005.0   3753     185              -25 -0.184877       -1
64  2020-05-14  24005.0   4306     223              131 -0.006455       -1
66  2020-05-16  24005.0   4566     240              150 -0.017375       -1
69  2020-05-19  24005.0   4936     263              169 -0.076848       -1
82  2020-06-01  24005.0   6315     349              235 -0.227684       -1
